In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [4]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
train_size  = 4096
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epochs = 200

In [5]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [6]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [7]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [8]:
x_train, x_test = x_train.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [9]:
layers = tf.keras.layers

In [10]:
gaussian_initializer=tf.keras.initializers.GlorotNormal()

In [11]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [12]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-4
    elif epoch < 55:
        return 1e-2
    elif epoch < 105:
        return 1e-2
    else:
        return 1e-4

In [13]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback])

Train on 3686 samples, validate on 410 samples
Epoch 1/200
3686/3686 [==============================] - 4s 1ms/sample - loss: 2.3102 - accuracy: 0.0936 - val_loss: 2.3040 - val_accuracy: 0.1268
Epoch 2/200
3686/3686 [==============================] - 2s 473us/sample - loss: 2.3091 - accuracy: 0.0941 - val_loss: 2.3032 - val_accuracy: 0.1268
Epoch 3/200
3686/3686 [==============================] - 0s 36us/sample - loss: 2.3081 - accuracy: 0.0944 - val_loss: 2.3024 - val_accuracy: 0.1293
Epoch 4/200
3686/3686 [==============================] - 0s 34us/sample - loss: 2.3071 - accuracy: 0.0947 - val_loss: 2.3016 - val_accuracy: 0.1293
Epoch 5/200
3686/3686 [==============================] - 0s 33us/sample - loss: 2.3061 - accuracy: 0.0952 - val_loss: 2.3009 - val_accuracy: 0.1293
Epoch 6/200
3686/3686 [==============================] - 0s 31us/sample - loss: 2.2862 - accuracy: 0.1213 - val_loss: 2.2668 - val_accuracy: 0.2073
Epoch 7/200
3686/3686 [==============================] - 0s 33us/

In [16]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128])

tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test)

tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test)

==========NTK============
4/4 [==============================] - 1s 226ms/step - loss: 3.7244 - accuracy: 0.1797
==========CE============
64/64 [==============================] - 0s 1ms/step - loss: 5.6641 - accuracy: 0.0723
==========MSE============
64/64 [==============================] - 0s 1ms/step - loss: 4.4032 - accuracy: 0.1240


[4.403232097625732, 0.1240234375]

In [22]:
model.evaluate(x_test, y_test)

64/64 [==============================] - 0s 1ms/step - loss: 2.2073 - accuracy: 0.4199


[2.207260847091675, 0.419921875]

In [16]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128])

4/4 [==============================] - 1s 215ms/step - loss: 2.8603 - accuracy: 0.3047


[2.8603286743164062, 0.3046875]